In [1]:
import json
import pandas as pd
import numpy as np
from scipy import stats
import itertools
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
from experiments.experiment_config import (
    ALL_LIGHTNING_MODELS,
    DRIVE_AND_ACT_VIEWPOINTS,
    # get_drive_and_act_loaders,
    # two_actors_samples,
    # four_actors_samples,
    # driver_and_act_pretrained_map,
)

sns.set_theme()

In [2]:
subset_percentage = [1, 2, 5, 10, 25, 50, 100]
actors = ['vp1', 'vp2', 'vp3', 'vp4', 'vp5', 'vp6', 'vp7', 'vp8']
line_colors = sns.color_palette('muted')[1:]
marker_colors = sns.color_palette('dark')[1:]

def calculate_summary_results(metric):    
    saved_folder = Path(f'saved_lifter_2d_3d_model/rq2/')
    view_point_mean = {}
    view_point_stderr = {}
    for view_point in DRIVE_AND_ACT_VIEWPOINTS:
        view_point_mean[view_point] = {}
        view_point_stderr[view_point] = {}
        for model in ALL_LIGHTNING_MODELS:
            model_name = model.__name__
            simple_model_name = model_name.replace('Lit', '')
            raw_model_mean = []
            raw_model_stderr = []
            ft_model_mean = []
            ft_model_stderr = []
            improv_model_mean = []
            improv_model_stderr = []
            for subset in subset_percentage:
                raw_measure = []
                ft_measure = []
                improv_measure = []
                for actor in actors:
                    raw_training_path = saved_folder / model_name / 'drive_and_act' / view_point / 'single_actor' / f'subset_{subset}' / actor
                    with (raw_training_path / 'test_result.json').open() as f:
                        data = json.loads(f.read())
                        raw_measure.append(pd.DataFrame(data[metric], index=[actor]))

                    transfer_learning_path = saved_folder / model_name / 'transfer_learning' / view_point / 'single_actor' / f'subset_{subset}' / actor
                    with (transfer_learning_path / 'test_result.json').open() as f:
                        data = json.loads(f.read())
                        ft_measure.append(pd.DataFrame(data[metric], index=[actor]))
                raw_model_mean.append(pd.concat(raw_measure).mean(axis=0))
                # raw_model_stderr.append(pd.DataFrame(stats.sem(pd.concat(raw_measure)), ))
                raw_model_stderr.append(pd.DataFrame(stats.sem(pd.concat(raw_measure)), index=raw_measure[0].columns, columns=['PJPE']))
                ft_model_mean.append(pd.concat(ft_measure).mean(axis=0))
                # ft_model_stderr.append(stats.sem(pd.concat(ft_measure)))
                ft_model_stderr.append(pd.DataFrame(stats.sem(pd.concat(ft_measure)), index=ft_measure[0].columns, columns=['std_error']))
                
                for i in range(len(raw_measure)):
                    improv_measure.append(raw_measure[i] - ft_measure[i])
                improv_model_mean.append(pd.concat(improv_measure).mean(axis=0))
                improv_model_stderr.append(pd.DataFrame(stats.sem(pd.concat(improv_measure)), index=improv_measure[0].columns, columns=['std_error']))

            raw_model_mean_df = pd.concat(raw_model_mean, axis=1)
            raw_model_mean_df.columns = subset_percentage
            ft_model_mean_df = pd.concat(ft_model_mean, axis=1)
            ft_model_mean_df.columns = subset_percentage
            improv_model_mean_df = pd.concat(improv_model_mean, axis=1)
            improv_model_mean_df.columns = subset_percentage

            raw_model_stderr_df = pd.concat(raw_model_stderr, axis=1)
            raw_model_stderr_df.columns = subset_percentage
            ft_model_stderr_df = pd.concat(ft_model_stderr, axis=1)
            ft_model_stderr_df.columns = subset_percentage
            improv_model_stderr_df = pd.concat(improv_model_stderr, axis=1)
            improv_model_stderr_df.columns = subset_percentage

            view_point_mean[view_point][simple_model_name] = {
                'raw': raw_model_mean_df,
                'ft': ft_model_mean_df,
                'improv': improv_model_mean_df,
            }
            view_point_stderr[view_point][simple_model_name] = {
                'raw': raw_model_stderr_df,
                'ft': ft_model_stderr_df,
                'improv': improv_model_stderr_df
            }

    return view_point_mean, view_point_stderr

In [3]:
def plot_result(
        metric,
        model,
        view_point,
        view_point_mean,
        view_point_stderr,
        markersize=3,
        elinewidth=1,
        capsize=3,
        legend_font_size=8,
        color_idx=0
    ):
    fig = plt.figure(figsize=(12, 35), constrained_layout = True)
    fig.suptitle(f'{metric} | view: {view_point}')
    model_name = model.__name__
    simple_model_name = model_name.replace('Lit', '')
    item_length = view_point_mean[view_point][simple_model_name]['raw'].index.shape[0]
    subfigs = fig.subfigures(item_length, 1, wspace=0.07)
    for i in range(item_length):
        axes = subfigs[i].subplots(1, 2)
        subfigs[i].suptitle(view_point_mean[view_point][simple_model_name]['raw'].index[i])
        axes[0].set_ylabel('MPJPE [mm]')
        axes[0].set_xlabel('Percentage of samples')
        axes[0].set(xticks=subset_percentage)
        axes[0].set_ylim(0, 150)
        raw_values = view_point_mean[view_point][simple_model_name]['raw'].iloc[i]
        raw_stderr = view_point_stderr[view_point][simple_model_name]['raw'].iloc[i]
        axes[0].plot(
            subset_percentage,
            raw_values,
            color=line_colors[color_idx],
            linestyle='dashed',
            label='No Pretrained'
        )
        axes[0].errorbar(
            subset_percentage,
            raw_values,
            yerr=raw_stderr,
            fmt='s',
            color=marker_colors[color_idx],
            elinewidth=elinewidth,
            capsize=capsize,
            markersize=markersize,
            label='Standard Error'
        )
        ft_values = view_point_mean[view_point][simple_model_name]['ft'].iloc[i]
        ft_stderr = view_point_stderr[view_point][simple_model_name]['ft'].iloc[i]
        axes[0].plot(
            subset_percentage,
            ft_values,
            color=line_colors[color_idx],
            label='Pretrained'
        )
        axes[0].errorbar(
            subset_percentage,
            ft_values,
            yerr=ft_stderr,
            fmt='o',
            elinewidth=elinewidth,
            capsize=capsize,
            markersize=markersize,
            color=marker_colors[color_idx],
            label='Standard Error'
        )
        axes[0].legend(prop={'size': legend_font_size})

        improv_values = view_point_mean[view_point][simple_model_name]['improv'].iloc[i]
        improv_stderr = view_point_stderr[view_point][simple_model_name]['improv'].iloc[i]

        axes[1].step(
            subset_percentage,
            improv_values,
            color=line_colors[color_idx],
            label='Reduction by pretraining',
            where='post'
        )
        axes[1].errorbar(
            subset_percentage,
            improv_values,
            yerr=improv_stderr,
            fmt='o',
            elinewidth=elinewidth,
            capsize=capsize,
            markersize=markersize,
            color=marker_colors[color_idx],
            label='Standard Error'
        )
        axes[1].set_ylabel('MPJPE [mm]')
        axes[1].set_xlabel('Percentage of samples')
        axes[1].set(xticks=subset_percentage)
        axes[1].set_ylim(0, 30)
        axes[1].legend(prop={'size': legend_font_size})
    plt.savefig(f'{metric}_{view_point}')

In [4]:
def get_summarize_table(view_point, simple_model_name, view_point_mean, sort_index=False):
    columns = view_point_mean['a_column_co_driver']['JointFormer']['raw'].index.tolist()
    if sort_index:
        columns = sorted(columns)
    columns = list(itertools.chain(*([[f'{i}', f'{i} (pretrain)'] for i in columns])))
    df = pd.concat(
        [
            view_point_mean[view_point][simple_model_name]['raw'].T,
            view_point_mean[view_point][simple_model_name]['ft'].T.add_suffix(' (pretrain)') 
        ], axis=1
    )[columns].T
    return df

def print_table(df):
    with pd.option_context('float_format', '{:.2f}'.format, 'display.max_row', None):
        display(df)

In [5]:
view_point_mean, view_point_stderr = calculate_summary_results('pjpe')
print_table(get_summarize_table(DRIVE_AND_ACT_VIEWPOINTS[0], ALL_LIGHTNING_MODELS[0].__name__.replace('Lit', ''), view_point_mean))

,1,2,5,10,25,50,100
nose,408.17,193.63,73.60,56.64,57.60,52.93,51.82
nose (pretrain),59.34,51.97,50.86,48.85,51.38,50.32,49.25
left_eye,380.89,194.71,74.05,54.50,57.17,50.89,49.15
left_eye (pretrain),53.56,51.66,48.81,47.44,49.35,48.15,47.05
right_eye,346.96,212.24,74.57,51.63,53.11,49.19,47.93
right_eye (pretrain),54.77,48.52,47.10,45.77,47.66,46.80,45.77
left_ear,149.75,76.49,35.80,21.28,17.98,15.17,14.43
left_ear (pretrain),17.08,14.93,14.55,13.19,13.38,13.73,13.80
right_ear,391.94,201.46,69.90,43.41,44.07,38.57,38.57
right_ear (pretrain),39.32,37.30,38.41,36.77,36.53,35.95,36.16


In [6]:
view_point_mean, view_point_stderr = calculate_summary_results('activity_mpjpe')
print_table(get_summarize_table(DRIVE_AND_ACT_VIEWPOINTS[0], ALL_LIGHTNING_MODELS[0].__name__.replace('Lit', ''), view_point_mean, sort_index=True))

,1,2,5,10,25,50,100
closing_bottle,416.24,235.59,109.42,88.09,85.93,86.00,84.55
closing_bottle (pretrain),90.27,78.82,83.79,83.89,82.52,83.42,82.57
closing_door_inside,514.36,305.78,161.10,141.11,138.71,137.17,138.85
closing_door_inside (pretrain),139.64,138.62,141.54,137.61,130.59,132.43,131.89
closing_laptop,428.25,246.85,137.03,121.27,121.60,119.80,120.72
closing_laptop (pretrain),129.89,116.68,126.64,124.68,119.35,117.92,116.55
drinking,394.47,216.11,100.38,79.34,77.46,78.01,77.45
drinking (pretrain),85.00,79.86,73.10,73.34,74.61,74.06,74.01
eating,357.69,197.84,87.39,67.52,67.52,64.60,62.86
eating (pretrain),76.28,66.98,65.20,64.17,63.28,62.51,60.88
